In [15]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [16]:
# read data
df = pd.read_csv('shopping_behavior_updated.csv')

In [17]:
#rating sunburst plt
ratings_data = df.groupby(['Category', 'Item Purchased'])['Review Rating'].agg(['mean', 'count']).reset_index()

#preprocess and prepare data for figure
ratings_sunburst = []
rating_min = ratings_data['mean'].min()
rating_max = ratings_data['mean'].max()

category_ratings = df.groupby('Category')['Review Rating'].agg(['mean', 'count']).reset_index()
for category in ratings_data['Category'].unique():
    cat_rating = category_ratings[category_ratings['Category'] == category]['mean'].values[0]
    ratings_sunburst.append(dict(
        parent="",
        id=category,
        name=category,
        value=ratings_data[ratings_data['Category'] == category]['count'].sum(),
        rating=cat_rating 
    ))
    category_items = ratings_data[ratings_data['Category'] == category]
    for _, row in category_items.iterrows():
        ratings_sunburst.append(dict(
            parent=category,
            id=f"{category}-{row['Item Purchased']}",
            name=row['Item Purchased'],
            value=row['count'],
            rating=row['mean']
        ))

# prepare for fig
fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=[item['id'] for item in ratings_sunburst],
    labels=[item['name'] for item in ratings_sunburst],
    parents=[item['parent'] for item in ratings_sunburst],
    values=[item['value'] for item in ratings_sunburst],
    branchvalues="total",
    marker=dict(
        colors=[item.get('rating', None) for item in ratings_sunburst],
        colorscale='RdYlGn',
        cmin=rating_min,
        cmax=rating_max,
        showscale=True,
        colorbar=dict(
            title="Rating",
            thickness=20,
            len=0.5,
            tickformat='.2f'
        )
    ),
    hovertemplate="""
    <b>%{label}</b><br>
    Reviews: %{value}<br>
    Rating: %{color:.2f}<br>
    <extra></extra>
    """
))

fig.update_layout(
    title={
        'text': "Product Ratings by Category and Item",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    width=1000,
    height=1000,
    annotations=[dict(
        x=0.5,
        y=-0.1,
        text="Inner ring: Categories<br>Outer ring: Items<br>Color intensity: Rating",
        showarrow=False,
        xref='paper',
        yref='paper',
        align='center'
    )]
)

fig.show()
fig.write_image('rating_category_sunburst.png')

In [18]:
# second plt Purchase Quantities

purchase_data = df.groupby(['Category', 'Item Purchased']).agg({
    'Customer ID': 'count'  
}).reset_index()

purchase_sunburst = []
category_totals = purchase_data.groupby('Category')['Customer ID'].sum()
max_category_total = category_totals.max()

for category in purchase_data['Category'].unique():
    purchase_sunburst.append(dict(
        parent="",
        id=category,
        name=category,
        value=purchase_data[purchase_data['Category'] == category]['Customer ID'].sum(),
        category_total=category_totals[category],

    ))

    category_items = purchase_data[purchase_data['Category'] == category]
    for _, row in category_items.iterrows():
        purchase_sunburst.append(dict(
            parent=category,
            id=f"{category}-{row['Item Purchased']}",
            name=row['Item Purchased'],
            value=row['Customer ID'],
            purchases=row['Customer ID']
        ))


fig_purchases = go.Figure()

fig_purchases.add_trace(go.Sunburst(
    ids=[item['id'] for item in purchase_sunburst],
    labels=[item['name'] for item in purchase_sunburst],
    parents=[item['parent'] for item in purchase_sunburst],
    values=[item['value'] for item in purchase_sunburst],
    branchvalues="total",
    marker=dict(
        colors=colors,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(
            title="Number of Purchases (Items)",
            thickness=20,
            len=0.5,
            y=0.8
        )
    ),
    hovertemplate="""
    <b>%{label}</b><br>
    Purchases: %{value}<br>
    <extra></extra>
    """
))

#second colorbar for categories
fig_purchases.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(
        colorscale='Reds',
        showscale=True,
        colorbar=dict(
            title="Total Purchases (Categories) Normalized",
            thickness=20,
            len=0.5,
            y=0.2
        )
    ),
    showlegend=False
))

fig_purchases.update_layout(
    title={
        'text': "Number of Purchases by Category and Item",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    width=900,
    height=900,
    
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    annotations=[dict(
        x=0.5,
        y=-0.1,
        text="Inner ring: Categories <br>Outer ring: Items<br>Color intensity: Number of Purchases",
        showarrow=False,
        xref='paper',
        yref='paper',
        align='center'
    )]
)

fig_purchases.show()
fig_purchases.write_image('counts_category_sunburst.png')

